In [ ]:
import pandas as pd

# Matching against previous injected data
df_waves = pd.concat([
    pd.read_csv('./previous_waves/wave-1/2025-09-11-production.csv', on_bad_lines='skip', sep=';', dtype=str, engine="c", keep_default_na=False, encoding="utf-8"),
    pd.read_csv('./previous_waves/wave-2/2025-09-19-production.csv', on_bad_lines='skip', sep=';', dtype=str, engine="c", keep_default_na=False, encoding="utf-8"),
    pd.read_csv('./previous_waves/wave-3/2025-09-26-production.csv', on_bad_lines='skip', sep=';', dtype=str, engine="c", keep_default_na=False, encoding="utf-8"),
])

In [ ]:
df_codes_to_generate = pd.read_csv('./codes-to-generate.csv', on_bad_lines='skip', sep=';', dtype=str, engine="c", encoding="utf-8")
df_codes_to_generate = df_codes_to_generate.merge(df_waves, how='inner', on='id_psp')

In [ ]:
assert(len(df_codes_to_generate) > 0)

In [ ]:
import requests

def get_pdf(row):
    payload = {
        'firstname': row['prenom'],
        'lastname': row['nom'],
        'dob': '2025-09-09',
        'gender': row['genre'],
        'code': row['id_psp']
    }
    response = requests.post(url, data=payload)
    filename = f"code-pass-sport-{row['nom']}-{row['prenom']}.pdf"

    if response.status_code == 200:
      output_path = f'./generated-pdfs/{filename}'
      with open(output_path, 'wb') as f:
        f.write(response.content)
      print(f"✅ Successfully downloaded PDF and saved to: {output_path}")
    else:
      print(f"❌ Failed to download PDF. Status code: {response.status_code}")
      try:
        print(f"Server response text: {response.json()}")
      except requests.exceptions.JSONDecodeError:
        print(f"Server response text: {response.text[:100]}...") # Print first 100 chars

url = 'http://localhost:3000/v2/api/eligibility-test/generate-pdf'
df_codes_to_generate.apply(lambda row: get_pdf(row), axis=1)